In [5]:
!pip3 install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3
    Uninstalling pip-23.3:
      Successfully uninstalled pip-23.3


In [4]:
!pip3 install -q tensorflow-recommenders
!pip3 install -q --upgrade tensorflow-datasets


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip


In [6]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

2023-12-16 23:37:30.934434: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:02<00:00,  1.92s/ url]

Dl Completed...: 100%|██████████| 1/1 [00:02<00:00,  1.92s/ url]

Dl Completed...: 100%|██████████| 1/1 [00:02<00:00,  1.92s/ url]

Dl Completed...: 100%|██████████| 1/1 [00:02<00:00,  1.92s/ url]

Dl Completed...: 100%|██████████| 1/1 [00:02<00:00,  2.88s/ url]


Dataset movielens downloaded and prepared to /Users/sushantjoshi/tensorflow_datasets/movielens/100k-ratings/0.1.1. Subsequent calls will reuse this data.


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 19.27 url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Dl Completed...: 100%|██████████| 1/1 [00:00<00:00, 13.68 url/s]
                                                                        

Dataset movielens downloaded and prepared to /Users/sushantjoshi/tensorflow_datasets/movielens/100k-movies/0.1.1. Subsequent calls will reuse this data.


In [20]:
for x in ratings.take(1).as_numpy_iterator():
  print(x)

{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)", 'user_id': b'138'}


2023-12-17 21:02:48.909123: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
2023-12-17 21:02:48.910104: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]


In [11]:
# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

In [18]:
movies.take(20)

<_TakeDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

## a recommendation system using TensorFlow Recommenders (TFRS)## 
It's creating vocabularies for user IDs and movie titles, which are essential steps in preparing data for a recommendation model.

In the first two lines, a `StringLookup` layer is created for user IDs. `StringLookup` is a layer that maps strings from a vocabulary to integer indices. The `mask_token=None` argument means that there's no mask token associated with this layer. A mask token is a special token that's not part of the vocabulary and is typically used to denote missing values or padding. By setting it to `None`, we're saying that there's no such special token in our data.

The `adapt` method is then called on the `user_ids_vocabulary` object with `ratings.map(lambda x: x["user_id"])` as the argument. This is using the `map` function to apply a lambda function to every element in the `ratings` dataset. The lambda function simply extracts the `"user_id"` field from each element. The `adapt` method adjusts the state of the `StringLookup` layer to reflect the data it's been given. In this case, it's building a vocabulary of unique user IDs.

The next two lines do the same thing for movie titles. A `StringLookup` layer is created and then adapted to the `movies` dataset. This builds a vocabulary of unique movie titles. The `movies` dataset is assumed to be a collection of movie titles.

In a recommendation system, these vocabularies are used to convert user IDs and movie titles into a format that the model can work with. The model will predict ratings based on these integer indices, and then the indices can be converted back into user IDs and movie titles for interpretation.

In [14]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

2023-12-16 23:45:36.880266: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]
2023-12-16 23:45:36.881563: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-12-16 23:46:32.232693: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{no

In [15]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [16]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [17]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

2023-12-16 23:47:29.700106: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
2023-12-16 23:47:29.701287: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


Epoch 1/3


2023-12-16 23:47:32.639024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_32' with dtype float and shape [4096,64]
	 [[{{node Placeholder/_32}}]]
2023-12-16 23:47:32.640348: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_32' with dtype float and shape [4096,64]
	 [[{{node Placeholder/_32}}]]


 1/25 [>.............................] - ETA: 1:15 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 9.7656e-04 - factorized_top_k/top_100_categorical_accuracy: 0.0151 - loss: 34069.6172 - regularization_loss: 0.0000e+00 - total_loss: 34069.6172

2023-12-16 23:47:33.080218: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]
2023-12-16 23:47:33.082446: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_32' with dtype float and shape [4096,64]
	 [[{{node Placeholder/_32}}]]


...

24/25 [===========================>..] - ETA: 0s - factorized_top_k/top_1_categorical_accuracy: 9.2570e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0091 - factorized_top_k/top_10_categorical_accuracy: 0.0236 - factorized_top_k/top_50_categorical_accuracy: 0.1460 - factorized_top_k/top_100_categorical_accuracy: 0.2707 - loss: 31982.1220 - regularization_loss: 0.0000e+00 - total_loss: 31982.1220

2023-12-16 23:48:14.255963: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_26' with dtype resource
	 [[{{node Placeholder/_26}}]]
2023-12-16 23:48:14.257375: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_32' with dtype float and shape [1696,64]
	 [[{{node Placeholder/_32}}]]


25/25 [==============================] - 17s 690ms/step - factorized_top_k/top_1_categorical_accuracy: 9.2000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0091 - factorized_top_k/top_10_categorical_accuracy: 0.0236 - factorized_top_k/top_50_categorical_accuracy: 0.1460 - factorized_top_k/top_100_categorical_accuracy: 0.2711 - loss: 30423.0931 - regularization_loss: 0.0000e+00 - total_loss: 30423.0931


2023-12-16 23:48:14.564287: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-12-16 23:48:14.566680: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_26' with dtype resource
	 [[{{node Placeholder/_26}}]]


Top 3 recommendations for user 42: [b'Rent-a-Kid (1995)' b'House Arrest (1996)' b'Just Cause (1995)']
